<a href="https://colab.research.google.com/github/mayibongwemoyo/dawm/blob/main/multi_method_incremental_watermarking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture

import sys
!pip install torchaudio soundfile matplotlib scipy datasets pandas seaborn
!pip install audioseal

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import sys
sys.path.append('content/drive/MyDrive/dawm/examples')
import noteb00k
from datasets import load_dataset
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torchaudio
import numpy as np
from scipy import stats
from audioseal import AudioSeal


# Constants
NUM_AUDIOS_PER_METHOD = 10
NUM_WATERMARKS = 4
SAMPLE_RATE = 16000

# Initialize models
generator = AudioSeal.load_generator("audioseal_wm_16bits")
detector = AudioSeal.load_detector("audioseal_detector_16bits")

ModuleNotFoundError: No module named 'noteb00k'

In [ ]:
def embed_sfa(audio, sr, alpha=0.3):
    """Sequential Fixed Alpha"""
    watermarked = audio.clone()
    metrics = []
    for i in range(NUM_WATERMARKS):
        watermarked = generator(watermarked, sr, alpha=alpha)
        metrics.append(calculate_metrics(audio, watermarked))
    return watermarked, metrics

def embed_sda(audio, sr):
    """Sequential Decaying Alpha"""
    watermarked = audio.clone()
    metrics = []
    for i in range(NUM_WATERMARKS):
        alpha = 0.5 / (i + 1)
        watermarked = generator(watermarked, sr, alpha=alpha)
        metrics.append(calculate_metrics(audio, watermarked))
    return watermarked, metrics

def embed_pfb(audio, sr, alpha=0.2):
    """Parallel Frequency Bands"""
    # Split into 4 frequency bands
    fft = torch.fft.fft(audio)
    bands = torch.chunk(fft, 4, dim=-1)

    # Embed in each band
    watermarked_bands = []
    for band in bands:
        wm_band = generator(band.real, sr, alpha=alpha) + 1j*band.imag
        watermarked_bands.append(wm_band)

    # Reconstruct
    watermarked = torch.fft.ifft(torch.cat(watermarked_bands, dim=-1)).real
    return watermarked, [calculate_metrics(audio, watermarked)]*4

In [ ]:
def calculate_metrics(original, watermarked):
    noise = watermarked - original
    snr = 10 * torch.log10(original.pow(2).mean() / noise.pow(2).mean()).item()

    # Test detection
    msg = torch.randint(0, 2, (1, 16))
    detector.message = msg
    prob, detected = detector.detect_watermark(watermarked, sr)
    ber = (msg != detected.round()).float().mean().item()

    return {
        "snr": snr,
        "ber": ber,
        "detection_prob": prob.item() if hasattr(prob, 'item') else prob
    }

In [ ]:
def process_batch(method_func, audios):
    results = []
    for audio, sr in audios[:NUM_AUDIOS_PER_METHOD]:
        audio = preprocess_audio(audio, sr)
        _, metrics = method_func(audio, SAMPLE_RATE)
        results.extend(metrics)
    return pd.DataFrame(results)

In [ ]:
# Load dataset
dataset = load_dataset("facebook/voxpopuli", "en", split="validation", streaming=True)
audios = [(torch.tensor(e["audio"]["array"]).float(), e["audio"]["sampling_rate"]) for e in dataset]

# Process all methods
methods = {
    "SFA": embed_sfa,
    "SDA": embed_sda,
    "PFB": embed_pfb
}

results = {name: process_batch(func, audios) for name, func in methods.items()}

In [ ]:
# ANOVA test for SNR differences
f_val, p_val = stats.f_oneway(
    results["SFA"]["snr"],
    results["SDA"]["snr"],
    results["PFB"]["snr"]
)
print(f"SNR ANOVA: F={f_val:.2f}, p={p_val:.4f}")

# Pairwise t-tests for BER
from itertools import combinations
for (m1, m2) in combinations(methods.keys(), 2):
    t, p = stats.ttest_ind(results[m1]["ber"], results[m2]["ber"])
    print(f"{m1} vs {m2} BER: t={t:.2f}, p={p:.4f}")

In [ ]:
def plot_metric(metric, ylabel):
    plt.figure(figsize=(10, 6))
    sns.lineplot(
        data=pd.concat({k: v[metric] for k, v in results.items()}, axis=1).melt(),
        x="variable", y="value", hue="Method",
        marker="o", ci=95
    )
    plt.title(f"Comparison of {metric.upper()} Across Methods")
    plt.xlabel("Watermark Index")
    plt.ylabel(ylabel)
    plt.show()

plot_metric("snr", "SNR (dB)")
plot_metric("ber", "Bit Error Rate")
plot_metric("detection_prob", "Detection Probability")

In [ ]:
# Plot spectrograms
def plot_spectrogram(audio, title):
    plt.specgram(audio.numpy().squeeze(), Fs=SAMPLE_RATE)
    plt.title(title)
    plt.show()

for method in methods:
    example_audio, _ = methods[method](audios[0][0], SAMPLE_RATE)
    plot_spectrogram(example_audio, f"{method} Watermarked Audio")

In [ ]:
from IPython.display import Audio

def play_audio(audio, sr):
    display(Audio(audio.numpy().squeeze(), rate=sr))

print("Original Audio:")
play_audio(audios[0][0], SAMPLE_RATE)

for method in methods:
    example_audio, _ = methods[method](audios[0][0], SAMPLE_RATE)
    print(f"{method} Watermarked Audio:")
    play_audio(example_audio, SAMPLE_RATE)